# Plaidbot Training

This notebook is for training a plaidbot model

In [1]:
# Imports
from app.options.model_options import ModelOptions
from app.options.prepro_options import PreproOptions
from app.src.pick_users.run_pick_users import run_pick_users
from app.src.select_data.run_select_data import run_select_data
from app.src.train.run_training import run_training
from app.src.predict.run_prediction import run_prediction

In [ ]:
# Ideally I'd like to just have this automatically save the results to the appropriate options...
# No other work needed on the users part
# Otherwise, Once it's finished, need to update Options and restart the env
run_pick_users(PreproOptions())

In [ ]:
#[('U0VCXUTSA', 'ruhanga'), ('U0XGB2W68', 'Ardan Burton'), ('U0XGWDDSS', 'Matthieu Belanger'), ('U0XKPNRQQ', 'Bert'), ('U12RJA20L', 'ChubThumper') ]

In [4]:
messages = run_select_data(PreproOptions())
# May want to save the messages... could make this optional

In [5]:
messages = messages[0:10000]

In [6]:
f = messages[4]
uds = [x.user_int_id for x in messages]
print(set(uds))

{0, 1, 2, 3, 4}


In [7]:
model = run_training(messages, ModelOptions())
# Will need to save the model and some of its components

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.weight', 'pre_classi



 --- Epoch 1
...batch 0


KeyboardInterrupt: 

In [ ]:
test_messages = [
    'This is a test'
]

preds = run_prediction(model, test_messages, ModelOptions())